In [ ]:
import numpy as np
import mayfly as mf
import h5py
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os 
import sys
import json
import scipy.signal
import scipy.stats
import scipy.interpolate
import pickle as pkl
import scipy.optimize
from pathlib import Path

PATH = '/storage/home/adz6/group/project/'
RESULTPATH = os.path.join(PATH, 'results/mayfly')
PLOTPATH = os.path.join(PATH, 'plots/mayfly')
DATAPATH = os.path.join(PATH, 'datasets/kass')


# load data

In [ ]:
os.listdir(os.path.join(DATAPATH))

In [ ]:
#test_data = mf.data.MFDataset(os.path.join(DATAPATH, '210930_87_18595_energy_angle_test_slice8192.h5'))
kass_data = h5py.File(os.path.join(DATAPATH, '211116_grad_b_est_kass.h5'))

#beamforming_result = np.load(os.path.join(RESULTPATH,'211005_mf_84_100_template_ideal_scores_bottom_tri.npy'))

In [ ]:
print(kass_data['kass'].keys())
print(kass_data['meta'].keys())
#print(kass_data['meta']['energy'].shape)
print(kass_data['kass']['x'].shape)

# load meta data

In [ ]:
kass_metadata = {}
for i, key in enumerate(kass_data['meta'].keys()):
    kass_metadata[key] = kass_data['meta'][key][:]
    
kass_metadata = pd.DataFrame.from_dict(kass_metadata)

In [ ]:
print(kass_metadata[(kass_metadata['x_min'] == 0.04) & (kass_metadata['theta_min'] == 90.0)])

# plot guiding center x and y example

In [ ]:
example_ind = 5250
fsample = 2000e6
tmax_ind = 2048
guide_x = kass_data['kass']['guide_x'][example_ind, :]
guide_y = kass_data['kass']['guide_y'][example_ind, :]
time = np.arange(0, guide_x.size, 1) / fsample

print(time)
sns.set_theme(context='paper', style='whitegrid')
clist = sns.color_palette('deep')
fig = plt.figure(figsize=(21, 8))

ax1 = fig.add_subplot(1,2,1)
ax1.plot(time[0:tmax_ind], guide_x[0:tmax_ind], color=clist[0])

ax2 = fig.add_subplot(1,2,2)
ax2.plot(time[0:tmax_ind], guide_y[0:tmax_ind], color=clist[1])


# plot x and y example

In [ ]:
example_ind = 5250
fsample = 2000e6
tmax_ind = 2048
x = kass_data['kass']['x'][example_ind, :]
y = kass_data['kass']['y'][example_ind, :]
time = np.arange(0, x.size, 1) / fsample

sns.set_theme(context='paper', style='whitegrid')
clist = sns.color_palette('deep')
fig = plt.figure(figsize=(21, 8))

ax1 = fig.add_subplot(1,2,1)
ax1.plot(time[0:tmax_ind], x[0:tmax_ind], color=clist[0])

ax2 = fig.add_subplot(1,2,2)
ax2.plot(time[0:tmax_ind], y[0:tmax_ind], color=clist[1])


# plot guiding center r and theta example

In [ ]:
example_ind = 5250
fsample = 2000e6
tmax_ind = 3 * 8192


guide_x = kass_data['kass']['guide_x'][example_ind, :]
guide_y = kass_data['kass']['guide_y'][example_ind, :]
print(guide_x.size)
time = np.arange(0, guide_x.size, 1) / fsample

r = np.sqrt(guide_x ** 2 + guide_y ** 2)
theta = np.degrees(np.arctan2(guide_y, guide_x))

sns.set_theme(context='paper', style='whitegrid')
clist = sns.color_palette('deep')
fig = plt.figure(figsize=(21, 8))

ax1 = fig.add_subplot(1,2,1)
ax1.plot(time[0:tmax_ind], r[0:tmax_ind], color=clist[0])

ax2 = fig.add_subplot(1,2,2)
ax2.plot(time[0:tmax_ind], theta[0:tmax_ind], color=clist[1])

print(theta[-1], theta[0])

print(f'Estimated grad-b frequency: {np.radians(theta[-1] - theta[0]) / (time[-1] * 2 * np.pi)} Hz')
print(kass_metadata['x_min'].iloc[example_ind])
print(kass_metadata['theta_min'].iloc[example_ind])

# calculate the grad-b frequency for each simulation

In [ ]:
grad_b_frequency = np.zeros(kass_data['kass']['x'].shape[0])
fsample = 2000e6

for i in range(grad_b_frequency.size):
    
    guide_x = kass_data['kass']['guide_x'][i, :]
    guide_y = kass_data['kass']['guide_y'][i, :]
    time = np.arange(0, guide_x.size, 1) / fsample

    r = np.sqrt(guide_x ** 2 + guide_y ** 2)
    theta = np.degrees(np.arctan2(guide_y, guide_x))
    
    grad_b_frequency[i] = np.radians(theta[-1] - theta[0]) / (time[-1] * 2 * np.pi)
    

# plot radius-pitch angle heat map of grad-b frequencies

In [ ]:
radii = np.sort(kass_metadata['x_min'].array.unique())
angle = np.sort(kass_metadata['theta_min'].array.unique())

rad_grid, angle_grid = np.meshgrid(radii, angle)

gradb_grid = np.zeros(rad_grid.shape).flatten()
missing_pixel_vals = []
missing_pixel_inds = []
for i, pair in enumerate(zip(rad_grid.flatten(), angle_grid.flatten())):
    
    try:
        pair_index = kass_metadata[(kass_metadata['x_min'] == pair[0]) & (kass_metadata['theta_min'] == pair[1])].index
        gradb_grid[i] = grad_b_frequency[pair_index]
    except:
        #print(pair)
        missing_pixel_vals.append([pair[0], pair[1]])
        missing_pixel_inds.append([np.argwhere(radii == pair[0])[0,0], np.argwhere(angle == pair[1])[0,0]])
        pass
    
gradb_grid = gradb_grid.reshape(rad_grid.shape)
missing_pixel_vals = np.array(missing_pixel_vals)
missing_pixel_inds = np.array(missing_pixel_inds)

print(gradb_grid.shape)


In [ ]:
sns.set_theme(style='ticks')
#cmap = sns.color_palette('coolwarm', as_cmap=True)
#cmap = sns.color_palette('tab10',as_cmap=True)
cmap = sns.diverging_palette(130, 290, as_cmap=True)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(gradb_grid, cmap=cmap, aspect='auto', interpolation='none', vmin=-15000, vmax=15000, extent=(0, 0.05, 90, 84))
cbar = fig.colorbar(img)

pixel_x = 0.05 / 51
pixel_y = 6 / 121
#ax.plot(missing_pixels[:, 0]-pixel_x/2, missing_pixels[:, 1]-pixel_y/2, '.')
#print(missing_pixels[0, :])
#rectangles = [matplotlib.patches.Rectangle((missing_pixels[i, 0]-pixel_x, missing_pixels[i, 1]+pixel_y/2), pixel_x, pixel_y/2, color='red', linewidth=0)
#             for i in range(missing_pixels.shape[0])]

#plt.Rectangle(missing_pixels[0, :],1,1, color='red')
#print(rectangles.shape)
#ax.add_collection(matplotlib.collections.PatchCollection(rectangles, color='red'))


# nearest neighbor interpolation of broken pixels

In [ ]:
def interpolate(missing_inds, image):
    
    skipped_pixels = []
    
    for i in range(missing_inds.shape[0]):
        
        neighbor_x = []
        neighbor_y = []
        
        missing_x = missing_inds[i, 0]
        missing_y = missing_inds[i, 1]
        
        if missing_x + 1 < image.shape[1]:
            neighbor_x.append(missing_x + 1)
        if missing_x - 1 >= 0:
            neighbor_x.append(missing_x - 1)
        if missing_y + 1 < image.shape[0]:
            neighbor_y.append(missing_y + 1)
        if missing_y - 1 >= 0:
            neighbor_y.append(missing_y - 1)
            
        neighbor_x.append(missing_x)
        neighbor_y.append(missing_y)
            
        neighbor_grid_x, neighbor_grid_y = np.meshgrid(neighbor_x, neighbor_y)
        
        freq_list = []
        
        for j, pair in enumerate(zip(neighbor_grid_x.flatten(), neighbor_grid_y.flatten())):
            #print(pair, missing_inds[:, 0], missing_inds[:, 1])
            if not np.any(np.logical_and(missing_inds[:, 0] == pair[0], missing_inds[:, 1] == pair[1])):
                freq_list.append(image[pair[1], pair[0]])
                #print(freq_list)
                #input()
                
        if len(freq_list) == 0:
            skipped_pixels.append([missing_x, missing_y])
        else:
            image[missing_y, missing_x] = np.mean(freq_list)
    skipped_pixels = np.array(skipped_pixels)
    
    if skipped_pixels.shape[0] == 0:
        return image
    else:
        interpolate(skipped_pixels, image)
        
        
gradb_grid_interpolate = np.array(gradb_grid, copy=True)
        
interpolate(missing_pixel_inds, gradb_grid_interpolate)

In [ ]:
sns.set_theme(style='ticks', context='talk')
#cmap = sns.color_palette('coolwarm', as_cmap=True)
#cmap = sns.color_palette('tab10',as_cmap=True)
cmap = sns.diverging_palette(130, 290, as_cmap=True)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(gradb_grid_interpolate, cmap=cmap, aspect='auto', interpolation='none', vmin=-12000, vmax=12000, extent=(0, 0.05, 90, 84))
cbar = fig.colorbar(img, label='Grad-B Frequency (Hz)')

ax.set_xlabel('Radius')
ax.set_ylabel('Pitch Angle')

plt.tight_layout()
save_path = Path.home()/'group'/'project'/'plots'/'analysis'/'kassiopeia'
save_path.mkdir(parents=True, exist_ok=True)
name = '220303_gradb_frequency_map.png'
plt.savefig(save_path/name)


In [ ]:
name = '220303_grad_b_frequency_grid_radius_angle'

rad_grid, angle_grid = np.meshgrid(radii, angle)

np.savez(os.path.join(RESULTPATH, name), freq=gradb_grid_interpolate, rad_grid=rad_grid, angle_grid=angle_grid)